In [ ]:
import huggingface_hub
hf_token = '' # put your User Access Tokens here
# ابتدا login کنید
huggingface_hub.login(token=hf_token)

# سپس وضعیت ورود را بررسی کنید
!hf auth whoami

user:  AM-Nateghi


In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import prepare_model_for_kbit_training
import torch

cptk = "google/gemma-3-1b-pt"
from transformers import BitsAndBytesConfig
tokenizer = AutoTokenizer.from_pretrained(cptk)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
model = AutoModelForCausalLM.from_pretrained(
    cptk,
    quantization_config=bnb_config,
    device_map="auto",
    dtype=torch.float16,
    attn_implementation='eager'
)
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False # for Gradient checkpointing

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_conf = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias='none',
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_conf)

In [13]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={
    "train": "assets/qa_train.json",
    "test": "assets/qa_test.json"
})

def formatter(example):
    return tokenizer(
        f"پرسش: {example['input']}\nپاسخ: {example['output']}\n",
        truncation=True,
        max_length=1024,
        padding="max_length"
    )

tokenized_dataset = dataset.map(formatter)

Map: 100%|██████████| 48/48 [00:00<00:00, 487.68 examples/s]


In [14]:
torch.cuda.empty_cache()

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback

callbacks = [EarlyStoppingCallback(early_stopping_patience=8)]

training_args = TrainingArguments(
    output_dir="./gemma_qlora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=5,
    eval_steps=50,
    save_steps=50,
    save_total_limit=2,
    metric_for_best_model="loss",
    save_strategy='steps',
    eval_strategy="steps",
    gradient_checkpointing=True,
    load_best_model_at_end=True,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    args=training_args,
    callbacks=callbacks,
)
trainer.train()

Truncating eval dataset: 100%|██████████| 48/48 [00:00<00:00, 4689.98 examples/s]


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
100,5.089100,5.035881,8.299445,102400.000000,1.000000
200,4.132100,4.106813,8.139786,204800.000000,1.000000


KeyboardInterrupt: 